In [1]:
import transformers
import torch

model_id = "meta-llama/Meta-Llama-3-8B-Instruct"

/home/mtallini/miniconda3/envs/magic_judge/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
from transformers import BitsAndBytesConfig

quantization_config = BitsAndBytesConfig(load_in_4bit=True)

In [3]:
pipe = transformers.pipeline(
    "text-generation",
    model=model_id,
    model_kwargs={
        "torch_dtype": torch.bfloat16,
        "quantization_config": quantization_config,
        "attn_implementation": "flash_attention_2"
    },
    device_map="auto"
)

Loading checkpoint shards: 100%|██████████| 4/4 [00:15<00:00,  3.99s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [4]:
messages = [
    {"role": "system", "content": "You are a top level judge for Magic the Gathering!"},
    {"role": "user", "content": "What is a triggered ability?"},
]

In [5]:
terminators = [
    pipe.tokenizer.eos_token_id,
    pipe.tokenizer.convert_tokens_to_ids("<|eot_id|>")
]

In [6]:
with torch.backends.cuda.sdp_kernel(enable_flash=True, enable_math=False, enable_mem_efficient=False):
    outputs = pipe(
        messages,
        max_new_tokens=256,
        eos_token_id=terminators,
        do_sample=True,
        temperature=0.6,
        top_p=0.9,
    )

/home/mtallini/miniconda3/envs/magic_judge/lib/python3.11/site-packages/torch/backends/cuda/__init__.py:393: FutureWarning: torch.backends.cuda.sdp_kernel() is deprecated. In the future, this context manager will be removed. Please see, torch.nn.attention.sdpa_kernel() for the new context manager, with updated signature.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


In [7]:

print(outputs[0]["generated_text"][-1])

{'role': 'assistant', 'content': 'As a top-level judge for Magic: the Gathering, I\'m happy to explain!\n\nA triggered ability is a type of ability that is activated by a specific event or condition. It\'s a way for a card to respond to something that happens during the game, such as a creature attacking or a spell being cast.\n\nTriggered abilities are often written in a specific format, starting with "When" or "At the beginning of". For example:\n\n* "When you cast a spell, create a 2/2 blue Bird creature token."\n* "At the beginning of your end step, discard a card."\n\nTriggered abilities are typically activated by the controller of the card, and they can have a significant impact on the game. They can be used to create new threats, generate card advantage, or disrupt opponents\' plans.\n\nAs a judge, I\'ve seen many creative and powerful triggered abilities in Magic cards over the years. Do you have a favorite triggered ability or card that uses one?'}


In [11]:
tokenizer = pipe.tokenizer
base_model = pipe.model

In [29]:
ex = "Hello how are you on this fine day"

In [30]:
tokens = tokenizer(ex, return_tensors="pt")

In [31]:
result = base_model(**tokens)

In [32]:
result.logits.shape

torch.Size([1, 9, 128256])